# TRAFFIC SIGNS CLASSIFICATION USING LE-NET ARCHITECTURE IN KERAS


- In this case study, you have been provided with images of traffic signs and the goal is to train a Deep Network to classify them
- The dataset contains 43 different classes of images.
- Classes are as listed below:

  - ( 0, b'Speed limit (20km/h)') ( 1, b'Speed limit (30km/h)')
  - ( 2, b'Speed limit (50km/h)') ( 3, b'Speed limit (60km/h)')
  - ( 4, b'Speed limit (70km/h)') ( 5, b'Speed limit (80km/h)')
  - ( 6, b'End of speed limit (80km/h)') ( 7, b'Speed limit (100km/h)')
  - ( 8, b'Speed limit (120km/h)') ( 9, b'No passing')
  - (10, b'No passing for vehicles over 3.5 metric tons')
  - (11, b'Right-of-way at the next intersection') (12, b'Priority road')
  - (13, b'Yield') (14, b'Stop') (15, b'No vehicles')
  - (16, b'Vehicles over 3.5 metric tons prohibited') (17, b'No entry')
  - (18, b'General caution') (19, b'Dangerous curve to the left')
  - (20, b'Dangerous curve to the right') (21, b'Double curve')
  - (22, b'Bumpy road') (23, b'Slippery road')
  - (24, b'Road narrows on the right') (25, b'Road work')
  - (26, b'Traffic signals') (27, b'Pedestrians') (28, b'Children crossing')
  - (29, b'Bicycles crossing') (30, b'Beware of ice/snow')
  - (31, b'Wild animals crossing')
  - (32, b'End of all speed and passing limits') (33, b'Turn right ahead')
  - (34, b'Turn left ahead') (35, b'Ahead only') (36, b'Go straight or right')
  - (37, b'Go straight or left') (38, b'Keep right') (39, b'Keep left')
  - (40, b'Roundabout mandatory') (41, b'End of no passing')
  - (42, b'End of no passing by vehicles over 3.5 metric tons')

- The network used is called Le-Net that was presented by Yann LeCun
  http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf


Citation

J. Stallkamp, M. Schlipsing, J. Salmen, and C. Igel. The German Traffic Sign Recognition Benchmark: A multi-class classification competition. In Proceedings of the IEEE International Joint Conference on Neural Networks, pages 1453–1460. 2011.

@inproceedings{Stallkamp-IJCNN-2011,
author = {Johannes Stallkamp and Marc Schlipsing and Jan Salmen and Christian Igel},
booktitle = {IEEE International Joint Conference on Neural Networks},
title = {The {G}erman {T}raffic {S}ign {R}ecognition {B}enchmark: A multi-class classification competition},
year = {2011},
pages = {1453--1460}
}


Dataset: https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification/data


## IMPORT LIBRARIES AND DATASET


In [ ]:
%pip install --upgrade pip setuptools wheel tensorflow==2.13.0 protobuf==3.20.3 numpy seaborn scikit-learn visualkeras scikit-image pydot graphviz pillow

In [ ]:
#Disable all warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import glob
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory

## LOAD THE DATA


In [ ]:
traindata_gen = ImageDataGenerator(zoom_range=0.5, shear_range=0.8, horizontal_flip=True, rescale=1/255)

In [ ]:
DATASET_DIR = "./p5_traffic_sign_classification"
TRAIN_DIR = os.path.join(DATASET_DIR, "traffic_Data", "DATA")
TEST_DIR = os.path.join(DATASET_DIR, "traffic_Data", "TEST")
data_train_gen =traindata_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(224,224),
    batch_size=3,
    class_mode="categorical",
    color_mode='rgb',
    seed = 1234,
    shuffle = True
    )

In [ ]:
data_train_gen.class_indices

## TEST DATA


In [ ]:
all_images = []
for img_path in glob.glob(os.path.join(TEST_DIR, "*")):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = img_array / 255.0  # divide by 255.0 to get float values between 0 and 1 (Rescale)
    all_images.append(img_array)

In [ ]:
imshow(all_images[40])

## IMPORT THE MODEL


In [ ]:
ResNet50_model = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))

### MODEL ARCHITECTURE


In [ ]:
# to make this notebook's output stable across runs

def reset_graph(seed=41):
    tf.random.set_seed(seed)
    np.random.seed(seed) 

In [ ]:
import visualkeras
from PIL import ImageFont

visualkeras.layered_view(ResNet50_model, legend=True)

In [ ]:
ResNet50_model.summary()

In [ ]:
plot_model(ResNet50_model, to_file= 'ResNet50_model.png', show_shapes = True, show_layer_names=True)

In [ ]:
ResNet50_model.output

In [ ]:
# Our Tuning

f1=Flatten()(ResNet50_model.output)
final_layer=Dense(58,activation='Softmax')(f1)
final_layer

In [ ]:
model=Model(inputs=ResNet50_model.input,outputs=final_layer)
model.summary()

In [ ]:
plot_model(model, to_file= 'model.png', show_shapes = True, show_layer_names=True)

In [ ]:
visualkeras.layered_view(model, legend=True)

### MODEL TRAINING


In [ ]:
reset_graph(seed=9)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_training = model.fit_generator(data_train_gen, epochs=10)